In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
from google.colab import drive
import zipfile

# For Google Colab file operations
from google.colab import files


In [ ]:
# Clone the YOLOv5 repository and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

# Ensure you have access to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
Mounted at /content/drive


In [ ]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# Define paths
base_path = '/content/drive/MyDrive/Yolo_Kaggle'
train_images_path = os.path.join(base_path, 'images/train')
train_labels_path = os.path.join(base_path, 'labels/train')
augmented_images_path = os.path.join(base_path, 'images/train_augmented')
augmented_labels_path = os.path.join(base_path, 'labels/train_augmented')

# Create directories for augmented data
os.makedirs(augmented_images_path, exist_ok=True)
os.makedirs(augmented_labels_path, exist_ok=True)

# Define augmentation pipeline
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Blur(blur_limit=3, p=0.2),
    A.HueSaturationValue(p=0.2),
    A.Rotate(limit=15, p=0.5),
])

def augment_images():
    for image_name in tqdm(os.listdir(train_images_path)):
        if image_name.endswith('.jpg') or image_name.endswith('.png'):
            image_path = os.path.join(train_images_path, image_name)
            label_path = os.path.join(train_labels_path, os.path.splitext(image_name)[0] + '.txt')

            # Load image and labels
            image = cv2.imread(image_path)
            height, width, _ = image.shape
            with open(label_path, 'r') as file:
                labels = file.readlines()

            for i in range(5):  # Create 4 augmented images + 1 original
                if i == 0:
                    augmented_image = image
                else:
                    augmented = augmentations(image=image)
                    augmented_image = augmented['image']

                # Save augmented image
                augmented_image_name = f"{os.path.splitext(image_name)[0]}_aug{i}.jpg"
                augmented_image_path = os.path.join(augmented_images_path, augmented_image_name)
                cv2.imwrite(augmented_image_path, augmented_image)

                # Save augmented labels
                augmented_label_name = f"{os.path.splitext(image_name)[0]}_aug{i}.txt"
                augmented_label_path = os.path.join(augmented_labels_path, augmented_label_name)
                with open(augmented_label_path, 'w') as file:
                    for label in labels:
                        class_id, x_center, y_center, bbox_width, bbox_height = map(float, label.split())
                        file.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

# Run the augmentation function
augment_images()


100%|██████████| 701/701 [08:14<00:00,  1.42it/s]


In [ ]:
# Define paths
data_yaml_path = os.path.join(base_path, 'data.yaml')
output_dir = os.path.join(base_path, 'weights')

In [ ]:
# Train YOLOv5 model with data augmentation and a larger model
!python train.py --img 640 --batch 16 --epochs 50 --data {data_yaml_path} --weights yolov5m.pt --cache ram --project {output_dir} --name yolo_kaggle --hyp data/hyps/hyp.scratch-low.yaml

2024-06-25 19:29:19.252583: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 19:29:19.252632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 19:29:19.253969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=/content/drive/MyDrive/Yolo_Kaggle/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_sc

In [ ]:

# Test the trained model with a video
# Ensure you have your test video in Google Drive
video_path = '/content/drive/MyDrive/videotest.mp4'  # Change this to your actual video path
output_video_path = '/content/drive/MyDrive/Yolo_Kaggle/detected_video.mp4'

!python detect.py --weights /content/drive/MyDrive/Yolo_Kaggle/weights/best.pt --source {video_path} --conf 0.25 --save-txt --save-conf --save-crop --project /content/drive/MyDrive/Yolo_Kaggle --name detected_video --exist-ok


detect: weights=['/content/drive/MyDrive/Yolo_Kaggle/weights/best.pt'], source=/content/drive/MyDrive/videotest.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/Yolo_Kaggle, name=detected_video, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-331-gab364c98 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/4752) /content/drive/MyDrive/videotest.mp4: 384x640 (no detections), 47.0ms
video 1/1 (2/4752) /content/drive/MyDrive/videotest.mp4: 384x640 (no detections), 7.6ms
video 1/1 (3/4752) /content/drive/MyDrive/videotest.mp4: 384x640 (no detections), 8.0